In [ ]:
#Imports
from __future__ import division

import random
import numpy as np
import plotly.graph_objects as go

from IPython.display import Markdown as md

In [ ]:
##aquire the data with the following global variables

N = 100 			#people participate in the market
start_cash = 100.00	#the starting budget per person
rounds = 500		#investment round that the particpants will partake in

people = [[start_cash] for i in range(N)] #list of participants with their money history in a list (start with just 100)

if (N%2):
    print "please keep N even for smooth operations"

def func_bound_exp_p(mean):
    chance = 2
    while chance > 1:
        chance = np.random.exponential(mean)
    return chance
    
def func_inv_round(input_people): 
    '''
    a single round of investments applied on the people list, returns the modified version
    '''
    
    sN = range(N) #range of indexes of the people participating
    random.shuffle(sN) #shuffled indexes so a random 50% win (first half of the shuffled indexes), the other 50% loses

    # percentage market gain/loss this round, up to 50% win or loss 
    p = random.random() / 2 #for uniform volatility with a max of 50%
    
    for cnt, Pidx in enumerate(sN):
        if  cnt == int(N / 2) :
            p = p*-1 #if halfway shuffled list, the rest loses money        
        
        #personal profit is based on how many % of their wealth is invested #maybe kijken of normal dist beter werkt hier?
        #personal_p = p * (random.random()) #if uniform distribution is desired
        personal_p = p * func_bound_exp_p(0.4)  #if exponential with mean x is desired
        
        #appending the new balance to the list
        #input_people[Pidx].append(round(people[Pidx][-1] + personal_p*people[Pidx][-1],2)) #add new rounded entry to the persons money record
        input_people[Pidx].append(input_people[Pidx][-1] + personal_p*input_people[Pidx][-1]) #add new entry to the persons money record
        
    return input_people #return the list of people with new balances

def func_investments(r):
    '''
    runs r rounds of investments and plots them
    '''
    global people #altering the original variable
    for i in range(r):
        people = func_inv_round(people)
    
    fig = go.Figure()
    for i in range(N):
        fig.add_trace(go.Scatter(x = range(r), y = people[i]))

    fig.show()  


    
func_investments(rounds)

In [ ]:
def func_sorted_time(t):
    '''
    takes a time as input and returns the sorted list of wealth from htl
    '''
    money_at_t = sorted([people[i][t] for i in range(N)])[::-1]
    return money_at_t

def func_powerlaw_t(t, log=False):
    '''
    creates a (log-log) plot for supplied t
    '''
    money_at_t = func_sorted_time(t)
    fpt_plot = go.Figure()
    fpt_plot.add_trace(go.Scatter(x = range(N), y = money_at_t))
    if log:
        fpt_plot.update_layout(xaxis_type="log", yaxis_type="log")
    fpt_plot.show()

    
    
func_powerlaw_t(-1)
func_powerlaw_t(-1, log=True)

In [ ]:
def func_polyfit_t(t):
    '''
    creates a polyfit for t and returns the [a,b] and [error]
    '''
    data = func_sorted_time(t)
    x = np.array(range(len(data))) + 1 # Array with same size as data, starting from 1 to avoid pesky divide-by-zero errors
    polynomial_coefficients, residuals = np.polyfit(np.log(x),np.log(data), 1,full=True)[:2] # Returns coefficients of a polynomial of degree 1 (just a linear relation) with least square fit to data
    
    return polynomial_coefficients, residuals



polycoeffs, error = func_polyfit_t(-1) #run the function for the last timeslot
print "polynomial coefficients [a,b] : {}\n               residual/error : {}".format(polycoeffs, error)

In [ ]:
'''turn the previous polynomial coeffs into a nice latex formula'''

md("the best fitting power law for the log-log plot of the division of wealth is at the last t is:  \n  \n$$y = e^{} + x^{}$$".format(("{"+str(round(polycoeffs[1],3))+"}"), ("{"+str(round(polycoeffs[0],3))+"}")) )

In [ ]:
def func_plots_with_fit(t):
    data = func_sorted_time(t)
    
    polynomial = np.poly1d(polycoeffs)

    log_plot = go.Figure()
    log_plot.add_trace(go.Scatter(x = range(N), y = data))
    log_plot.add_trace(go.Scatter(x = range(N), y = np.exp(polynomial(np.log(range(N))))))
    log_plot.update_layout(xaxis_type="log", yaxis_type="log")
    log_plot.show()

    lin_plot = go.Figure()
    lin_plot.add_trace(go.Scatter(x = range(N), y = data))
    lin_plot.add_trace(go.Scatter(x = range(N), y = np.exp(polynomial(np.log(range(N))))))
    lin_plot.show()
    
    
    
func_plots_with_fit(-1)

In [ ]:
# Step 5 : plot a over time

a_over_time = []

for r in range(rounds):
    data_r = func_powerlaw_t(r)
    polycoeffs, error = polyfit_data(data_r)
    a_over_time.append(-1 * polycoeffs[0])
    
alpha_plot = go.Figure()
alpha_plot.add_trace(go.Scatter(x = range(r), y = a_over_time))

alpha_plot.show()

In [ ]:
#Step 6 : plot residuals over time

e_over_time = []

for r in range(rounds):
    data_r = func_powerlaw_t(r)
    polycoeffs, error = polyfit_data(data_r)
    e_over_time.append(error[0])
    
e_plot = go.Figure()
e_plot.add_trace(go.Scatter(x = range(r), y = e_over_time))

e_plot.show()

In [ ]:
#Step 7 : What % population own 50% of the wealth

def halfwaycash(ls_in, info = False):
    half_total_cash = sum(ls_in)/2 #what is halfway
    
    track_cash = 0
    wealthy = 0
    for idx, cash in enumerate(ls_in):
        if (abs(half_total_cash - track_cash) >
            abs(half_total_cash - (track_cash+cash))): #if the new itteration is closer to half than the previous
            track_cash += cash
        else:
            if info:
                print("\n\nthere are {} rich that closest split the graph 50/50 left and right with €{} combined".format(idx, round(track_cash,2)))
                print("the nr. {} most wealthy person has €{}".format(idx, round(ls_in[idx-1],2)))
            return (round(idx/N *100,2))
    return

p50_over_time = []

for r in range(rounds):
    data_r = func_powerlaw_t(r)
    p50_over_time.append(halfwaycash(data_r))
    
p50_plot = go.Figure()
p50_plot.add_trace(go.Scatter(x = range(r), y = p50_over_time))

p50_plot.show()




#halfwaycash(func_powerlaw_t(10), info = True)
    